In [4]:
import re
import spacy
import pandas as pd

In [24]:
df = pd.read_csv("./DicatCSVJuanRana_cleaned.csv")

textos = df["JUAN_RANA_cleaned"].tolist()

In [18]:
textos = ["Juan Rana formó parte de la compañía de Pedro de la Rosa que representó en el Corpus de Sevilla —Cotarelo, al tomar esta información de Sánchez Arjona, afirma que estuvo en Sevilla y Málaga, pero nada dice éste sobre una estancia en Málaga—. 1712 Francisca Borja representó quintas damas en la compañía de José Garcés en Madrid. 1717 Francisca Borja representó terceras damas en la compañía de Juan Álvarez en Madrid. Consta un concierto, fechado en Madrid el 13 de marzo, de Agustín de Cáceres, como padre de María Leonor de Cáceres, y de Jerónima de Zayas, soltera, y José de Porras por el que se comprometían a ir a Almonacid de Zurita a hacer la fiesta del Corpus, para lo cual dichas actrices irían quince días antes para ensayar, y José de Porras ocho días antes, corriendo los viajes a cargo del mayordomo, y recibiendo por ello María Leonor de Cáceres 700 rs., José de Porras 400 rs. y Jerónima de Zayas 300 rs., y cabalgaduras para todos ellos y para Agustina de Cáceres, madre de María Leonor y cabalgaduras para ella y para su madre, Agustina de Cáceres."]

In [22]:
nlp = spacy.load("es_core_news_md")

verbos_objetivo = ["formar parte de", "representar", "recibir", "comprometer a"]

relaciones = []

for texto in textos:
    doc = nlp(texto)
    for ent in doc.ents:
        if ent.label_ in ("PER", "ORG"):
            sujeto = ent.text.strip()
            
            # Lematizar los verbos
            oracion = ent.sent
            tokens = [token.lemma_ if token.pos_ == "VERB" else token.text for token in oracion]
            texto_lemas = " ".join(tokens)
    
    
            for verbo_esperado in verbos_objetivo:
                sujeto_regex = re.escape(sujeto)
                verbo_regex = re.escape(verbo_esperado)
    
                # Patrón personalizado por sujeto y verbo
                patron = fr"\b{sujeto_regex}\b(?:\W+\w+){{0,5}}\W+(?P<verbo>{verbo_regex})\b\s+(?P<objeto>[^.]+)"

    
                coincidencia = re.search(patron, texto_lemas)
                if coincidencia:
                    verbo = coincidencia.group("verbo").strip()
                    objeto_original = coincidencia.group("objeto").strip()
                    
                    # Reducción del objeto usando chunks nominales
                    objeto_final = objeto_original
                    for chunk in oracion.noun_chunks:
                        if chunk.root.lemma_ in objeto_original and chunk.root.pos_ != "PRON":
                            objeto_final = chunk.text
                            break
    
                    relaciones.append({
                        "sujeto": sujeto,
                        "verbo": verbo,
                        "objeto": objeto_final,
                        "contexto": oracion.text
                    })
    
                    break

# Mostrar resultados
print("\nRelaciones (Sujeto → Verbo → Objeto):\n")
for r in relaciones:
    print(f"- {r['sujeto']} → {r['verbo']} → {r['objeto']}")
    #print(f"- {r['sujeto']} → {r['verbo']} → {r['objeto']} ({r['contexto']})")



Relaciones (Sujeto → Verbo → Objeto):

